In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [6]:
data = pd.read_csv("../data/aggregated.csv")
data.dropna(inplace=True)
data.set_index("Date", inplace=True)
data.head()

,AAPL,AMZN,GOOG,IBM,INTC,META,ORCL,NVDA,MSFT,TSLA
Date,,,,,,,,,,
6/8/2021,74403800.0,68334000.0,25060000,5313785.0,0.0,0,10726800,0.0,0.0,0.0
6/9/2021,0.0,49110000.0,0,5547252.0,0.0,0,0,0.0,0.0,0.0
6/10/2021,0.0,69530000.0,31234000,0.0,26738100.0,13240200,0,28777200.0,24563600.0,71758800.0
6/11/2021,53522400.0,0.0,0,3596462.0,18119700.0,0,11066500,41630800.0,0.0,0.0
6/14/2021,96906500.0,51394000.0,22550000,0.0,21784500.0,12250800,0,32137600.0,19150500.0,61272000.0


In [9]:
baskets = data.applymap(lambda x : 1 if x > 0 else 0)
baskets.head()

,AAPL,AMZN,GOOG,IBM,INTC,META,ORCL,NVDA,MSFT,TSLA
Date,,,,,,,,,,
6/8/2021,1,1,1,1,0,0,1,0,0,0
6/9/2021,0,1,0,1,0,0,0,0,0,0
6/10/2021,0,1,1,0,1,1,0,1,1,1
6/11/2021,1,0,0,1,1,0,1,1,0,0
6/14/2021,1,1,1,0,1,1,0,1,1,1


In [11]:
frequent_itemsets = apriori(baskets, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2]
frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)
frequent_itemsets.head(10)

/home/codespace/.python/current/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets,length
17,0.440476,"(AAPL, MSFT)",2
11,0.424603,"(AAPL, GOOG)",2
16,0.410714,"(NVDA, AAPL)",2
32,0.404762,"(GOOG, MSFT)",2
14,0.402778,"(AAPL, META)",2
29,0.400794,"(GOOG, META)",2
10,0.398810,"(AAPL, AMZN)",2
47,0.392857,"(META, MSFT)",2
52,0.392857,"(NVDA, MSFT)",2
46,0.388889,"(NVDA, META)",2
